In [6]:
# install librosa if it is not avaliable 
! pip install librosa==0.7.2

     |████████████████████████████████| 1.6MB 10.9MB/s 
  Created wheel for librosa: filename=librosa-0.7.2-cp36-none-any.whl size=1612885 sha256=75d9e81a002b286f225326516b51c927874470362641430681622cab621dd882
  Stored in directory: /root/.cache/pip/wheels/4c/6e/d7/bb93911540d2d1e44d690a1561871e5b6af82b69e80938abef
Successfully built librosa
  Found existing installation: librosa 0.6.3
    Uninstalling librosa-0.6.3:
      Successfully uninstalled librosa-0.6.3


In [1]:
# Mount the predict-wearer from google drive
import os 
os.chdir('./drive/My Drive/predict-wearer')

In [2]:
from utilize.data import *
from utilize.model import *
from utilize.transform import *
import tensorflow.keras as keras
import numpy as np

In [3]:
LOG_FOLDER = './results/vgg_slimmer_0607_002_rl0.0001_dropout0.6_fl960_hl160_bs128/'
DATA_PATH = "./data/002_0528.wav"
LABELS_PATH = "./data/002_0528.txt"

In [4]:
if not os.path.exists(LOG_FOLDER): 
    os.mkdir(LOG_FOLDER)

In [5]:
# Load the data 
X, y, sr = load_mfcc_frames_multiclass(data_path = DATA_PATH, labels_path = LABELS_PATH)

In [6]:
X.shape, y.shape

((43413, 42, 12, 1), (43413, 3))

In [7]:
np.unique(y, return_counts=True, axis = 0)

(array([[False, False,  True],
        [False,  True, False],
        [ True, False, False]]), array([30984,  5648,  6781]))

In [8]:
X_balanced, y_balanced = balance_multi_classes(X, y)
X_train, y_train, X_test, y_test = train_test_split_multi_classes(X_balanced, y_balanced)

In [9]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((15252, 42, 12, 1), (15252, 3), (1692, 42, 12, 1), (1692, 3))

In [10]:
model = VGG_Slimmer(p = 0.6, num_output = 3)
model.compile(optimizer = keras.optimizers.RMSprop(learning_rate=0.0001),
                loss = 'categorical_crossentropy',
                metrics = ['categorical_accuracy'])

In [11]:
callback = tf.keras.callbacks.TensorBoard(log_dir = LOG_FOLDER, histogram_freq=1)

model.fit(X_train, y_train, 
          batch_size=128,  epochs=100, verbose = 1, 
          validation_data = (X_test, y_test), 
          callbacks = [callback, keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=20, mode='auto')])

Epoch 1/100
120/120 [==============================] - 3s 26ms/step - loss: 6.9447 - categorical_accuracy: 0.3349 - val_loss: 1.0787 - val_categorical_accuracy: 0.4356
Epoch 2/100
120/120 [==============================] - 2s 13ms/step - loss: 1.7798 - categorical_accuracy: 0.3608 - val_loss: 1.0678 - val_categorical_accuracy: 0.4521
Epoch 3/100
120/120 [==============================] - 2s 13ms/step - loss: 1.2061 - categorical_accuracy: 0.3799 - val_loss: 1.0335 - val_categorical_accuracy: 0.4504
Epoch 4/100
120/120 [==============================] - 2s 13ms/step - loss: 1.0581 - categorical_accuracy: 0.4586 - val_loss: 1.2935 - val_categorical_accuracy: 0.4504
Epoch 5/100
120/120 [==============================] - 2s 13ms/step - loss: 0.9849 - categorical_accuracy: 0.5028 - val_loss: 1.2069 - val_categorical_accuracy: 0.4811
Epoch 6/100
120/120 [==============================] - 2s 13ms/step - loss: 0.9405 - categorical_accuracy: 0.5241 - val_loss: 1.1919 - val_categorical_accuracy: